In [1]:
import pandas as pd
import plotly.express as px
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from jupyter_dash import JupyterDash
import plotly.graph_objects as go

df = pd.read_csv('project_dataset.csv')
df.dropna(subset=['date_start', 'date_end', 'medium_display', 'artwork_type_title', 'department_title', 'artist_title'],
          inplace=True)

df['colorfulness'].fillna(df['colorfulness'].median(), inplace=True)
font_t = {'family': 'Andale Mono', 'size': 20, 'weight': 'bold'}
font_lab = {'family': 'Andale Mono', 'size': 16}

df["date_start"] = df["date_start"].astype(int)
top_medium_types = df['medium_display'].value_counts().nlargest(5).index
df_filtered = df[df['medium_display'].isin(top_medium_types)]


DASH

In [2]:
TYPES = df_filtered['medium_display'].unique().tolist()
type_one_options = [ {"label": pktype, "value": pktype} for pktype in TYPES]

CTYPE = {'Color woodblock print; oban': '#b93252', 'Color woodblock print': '#4ba4b1', 'Etching on paper': '#5755a5', 'Lithograph on paper': '#fca85d', 'Gelatin silver print': '#a0d8a4'}

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css'] # define the html style. You can store it your device for offline tests.

In [3]:
app = JupyterDash(__name__, external_stylesheets = external_stylesheets)

app.layout = html.Div([
        html.Div([
            html.Div([
                html.P("Filter by Medium:", className="control_label"),
                dcc.Dropdown(
                    id="one_types",
                    options=type_one_options,
                    multi=True,
                    value=[
                        "Color woodblock print; oban",
                        "Color woodblock print",
                        "Etching on paper",
                        "Lithograph on paper",
                        "Gelatin silver print",
                    ],
                    className="dcc_control",
                ),
            ], className="pretty_container four columns", id="cross-filter-options"),
            html.Div([dcc.Graph(id="graph")], id="countGraphContainer", className="pretty_container twelve columns"),
        ], className="row flex-display"),
        html.Div([
            html.Div([dcc.Graph(id="scatterplot")], className="pretty_container twelve columns"),
        ], className="row flex-display"),
    ],
    id="mainContainer",
    style={"display": "flex", "flex-direction": "column"}
)



In [4]:
@app.callback(  Output("graph", "figure"),
                [Input('one_types', "value")])
def make_histogram(one_types):
    fig = px.histogram(df_filtered[df_filtered["medium_display"].isin(one_types)],
                       x="colorfulness",
                       color="medium_display",
                       color_discrete_map=CTYPE)

    fig.update_traces(textposition="outside")
    fig.update_layout(template='simple_white',
        title={
            'text': 'Colorfulness in Medium Displays',
            'font': {'size': 20, 'family': 'Andale Mono', 'color': 'black'},
            'x': 0.5, # Center title horizontally
            'y': 1  # Position title vertically
        },
        font={
            'family': 'Andale Mono',
            'size': 14,
            'color': 'black'
        },
        legend={
            'title': {'text': 'Medium Display', 'font': {'size': 16, 'family': 'Andale Mono', 'color': 'black'}},
            'font': {'size': 12, 'family': 'Andale Mono', 'color': 'black'}
        },
        xaxis={
            'title': {'text': 'Colorfulness', 'font': {'size': 14, 'family': 'Andale Mono', 'color': 'black'}},
            'tickfont': {'size': 12, 'family': 'Andale Mono', 'color': 'black'}
        },
        yaxis={
            'title': {'text': 'Count', 'font': {'size': 14, 'family': 'Andale Mono', 'color': 'black'}},
            'tickfont': {'size': 12, 'family': 'Andale Mono', 'color': 'black'}
        },
        uniformtext_minsize=10,
        uniformtext_mode="hide"
    )

    return fig

In [5]:
@app.callback(Output("scatterplot", "figure"),
              [Input("one_types", "value")])
def make_scatterplot(one_types):
    fig = px.scatter(df_filtered[df_filtered["medium_display"].isin(one_types)],
                     x="date_start",
                     y="colorfulness",
                     color="medium_display",
                     color_discrete_map=CTYPE)
    fig.update_traces(marker_size=8)
    fig.update_layout(template='simple_white',
        title={
            'text': 'Colorfulness Across the Years',
            'font': {'size': 20, 'family': 'Andale Mono', 'color':'black'},
            'x': 0.5, # Center title horizontally
            'y': 1  # Position title vertically
        },
        font={
            'family': 'Andale Mono',
            'size': 14,
            'color': 'black'
        },
        legend={
            'title': {'text': 'Medium Display', 'font': {'size': 15, 'family': 'Andale Mono', 'color': 'black'}},
            'font': {'size': 12, 'family': 'Andale Mono', 'color': 'black'}
        },
        xaxis={
            'title': {'text': 'Years', 'font': {'size': 14, 'family': 'Andale Mono', 'color': 'black'}},
            'tickfont': {'size': 12, 'family': 'Andale Mono', 'color': 'black'}
        },
        yaxis={
            'title': {'text': 'Colorfulness', 'font': {'size': 14, 'family': 'Andale Mono', 'color': 'black'}},
            'tickfont': {'size': 12, 'family': 'Andale Mono', 'color': 'black'}
        }
    )
    return fig

In [7]:
app.run_server(mode='inline', port=2345)

#app.save('dash-project.html')

Dash is running on http://127.0.0.1:2345/

